In [ ]:
# import json
# from aiida import load_profile
# from quantum_espresso_workflow import (
#     get_bulk_structure,
#     calculate_qe,
#     plot_energy_volume_curve,
# )

# # from python_workflow_definition.pyiron_base import  get_dict
# from aiida_workgraph import WorkGraph, task
# from typing import Any

# load_profile()


# # from python_workflow_definition.base
# @task.pythonjob()
# def get_list(**kwargs):
#     return list(kwargs.values())


# # from python_workflow_definition.base
# @task.pythonjob()
# def get_dict(**kwargs):
#     return {k: v for k, v in kwargs.items()}


# # -------------------------------------------------------------
# # These is the helper functions for the WorkGraph onlly
# @task.pythonjob()
# def pickle_node(value):
#     """Handle data nodes"""
#     return value


# get_bulk_structure_task = task.pythonjob()(get_bulk_structure)
# generate_structures_task = task.pythonjob()(generate_structures)
# calculate_qe_task = task.pythonjob(outputs=["energy", "volume", "structure"])(
#     calculate_qe
# )
# plot_energy_volume_curve_task = task.pythonjob()(plot_energy_volume_curve)

# strain_lst = [0.9, 0.95, 1.0, 1.05, 1.1]
# wg = WorkGraph()
# wg.add_task(
#     get_bulk_structure_task,
#     name="bulk",
#     element="Al",
#     a=4.05,
#     cubic=True,
#     register_pickle_by_value=True,
# )
# wg.add_task(pickle_node, name="calculation", value="vc-relax")
# wg.add_task(pickle_node, name="kpts", value=[3, 3, 3])
# wg.add_task(
#     pickle_node, name="pseudopotentials", value={"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}
# )
# wg.add_task(pickle_node, name="smearing", value=0.02)
# wg.add_task(
#     get_dict,
#     name="get_dict",
#     structure=wg.tasks.bulk.outputs.result,
#     calculation=wg.tasks.calculation.outputs.result,
#     kpts=wg.tasks.kpts.outputs.result,
#     pseudopotentials=wg.tasks.pseudopotentials.outputs.result,
#     smearing=wg.tasks.smearing.outputs.result,
#     register_pickle_by_value=True,
# )
# wg.add_task(
#     calculate_qe_task,
#     name=f"relax",
#     input_dict=wg.tasks.get_dict.outputs.result,
#     working_directory="mini",
#     register_pickle_by_value=True,
# )
# wg.add_task(
#     generate_structures_task,
#     name="generate_structures",
#     structure=wg.tasks.relax.outputs.structure,
#     strain_lst=strain_lst,
#     register_pickle_by_value=True,
# )
# # here we add the structure outputs based on the number of strains
# del wg.tasks.generate_structures.outputs["result"]
# for i in range(len(strain_lst)):
#     wg.tasks.generate_structures.add_output("workgraph.any", f"s_{i}")
# wg.add_task(get_list, name="get_energies", register_pickle_by_value=True)
# wg.add_task(get_list, name="get_volumes", register_pickle_by_value=True)
# wg.add_task(pickle_node, name="calculation_scf", value="scf")
# for i, strain in enumerate(strain_lst):
#     get_dict_task = wg.add_task(
#         get_dict,
#         name=f"get_dict_{i}",
#         calculation=wg.tasks.calculation_scf.outputs.result,
#         structure=wg.tasks.generate_structures.outputs[f"s_{i}"],
#         kpts=wg.tasks.kpts.outputs.result,
#         pseudopotentials=wg.tasks.pseudopotentials.outputs.result,
#         smearing=wg.tasks.smearing.outputs.result,
#         register_pickle_by_value=True,
#     )
#     qe_task = wg.add_task(
#         calculate_qe_task,
#         name=f"qe_{i}",
#         input_dict=get_dict_task.outputs.result,
#         working_directory=f"strain_{i}",
#         register_pickle_by_value=True,
#     )
#     # collect energy and volume
#     wg.add_link(qe_task.outputs.energy, wg.tasks.get_energies.inputs.kwargs)
#     wg.add_link(qe_task.outputs.volume, wg.tasks.get_volumes.inputs.kwargs)

# wg.add_task(
#     plot_energy_volume_curve_task,
#     volume_lst=wg.tasks.get_volumes.outputs.result,
#     energy_lst=wg.tasks.get_energies.outputs.result,
#     register_pickle_by_value=True,
# )
# wg
# # wg.to_html()


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [ ]:
from aiida import load_profile
from quantum_espresso_workflow import (
    generate_structures,
    get_bulk_structure,
    calculate_qe,
    plot_energy_volume_curve,
)

from python_workflow_definition.aiida import get_list, get_dict, pickle_node, construct_wg_qe

# from python_workflow_definition.pyiron_base import  get_dict
from aiida_workgraph import WorkGraph, task
from typing import Any

load_profile()

get_list_task = task.pythonjob(get_list)
get_dict_task = task.pythonjob(get_dict)
pickle_node_task = task.pythonjob(pickle_node)
get_bulk_structure_task = task.pythonjob()(get_bulk_structure)
generate_structures_task = task.pythonjob()(generate_structures)
calculate_qe_task = task.pythonjob(outputs=["energy", "volume", "structure"])(
    calculate_qe
)
plot_energy_volume_curve_task = task.pythonjob()(plot_energy_volume_curve)

strain_lst = [0.9, 0.95, 1.0, 1.05, 1.1]

wg = construct_wg_qe(
    get_dict_task=get_dict_task,
    get_list_task=get_list_task,
    pickle_node_task=pickle_node_task,
    get_bulk_structure_task=get_bulk_structure_task,
    calculate_qe_task=calculate_qe_task,
    generate_structures_task=generate_structures_task,
    plot_energy_volume_curve_task=plot_energy_volume_curve_task,
    strain_lst=strain_lst
)

wg
# wg.to_html()


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [3]:
wg.run()

/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/querybuilder/main.py:182: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  result = build.query.count()
03/17/2025 06:48:22 AM <3377989> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8137|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/17/2025 06:48:23 AM <3377989> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8137|WorkGraphEngine|continue_workgraph]: tasks ready to run: bulk,kpts,smearing,calculation,calculation_scf,pseudopotentials
03/17/2025 06:48:23 AM <3377989> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8137|WorkGraphEngine|run_tasks]: Run task: bulk, type: PYTHONJOB
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-p

------------------------------------------------------------
kwargs:  {'a': 4.05, 'cubic': True, 'element': 'Al', 'register_pickle_by_value': True}


/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/nodes.py:224: SAWarning: Object of type <DbNode> not in session, add operation along 'DbComputer.dbnodes' will not proceed
  session.commit()
03/17/2025 06:48:25 AM <3377989> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8137|WorkGraphEngine|run_tasks]: Run task: kpts, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': [3, 3, 3]}


/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/backend.py:271: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed
  with session.begin_nested() as savepoint:
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/utils.py:84: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  return getattr(self._model, item)
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/backend.py:271: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed
  with session.begin_nested() as savepoint:
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/nodes.py:224: SAWarning: Obje

...

/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/backend.py:271: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed
  with session.begin_nested() as savepoint:
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/utils.py:84: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  return getattr(self._model, item)
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/backend.py:271: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed
  with session.begin_nested() as savepoint:
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/nodes.py:224: SAWarning: Obje

------------------------------------------------------------
kwargs:  {'energy_lst': <List: uuid: 6966fa72-9f11-461d-a144-924d4016a218 (pk: 8399) value: [-1074.8457448491, -1074.916148965, -1074.9365241649, -1074.9192859118, -1074.8737903054]>, 'register_pickle_by_value': True, 'volume_lst': <List: uuid: dfa296a3-559e-46ff-b998-f21ea56cbb8a (pk: 8398) value: [59.594114001539, 62.904898112736, 66.215682223933, 69.526466335129, 72.837250446326]>}


/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/nodes.py:224: SAWarning: Object of type <DbNode> not in session, add operation along 'DbComputer.dbnodes' will not proceed
  session.commit()
03/17/2025 06:53:11 AM <3377989> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [8137|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 8406
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/utils.py:84: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  return getattr(self._model, item)
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/aiida/storage/psql_dos/orm/utils.py:84: SAWarning: Object of type <DbNode> not in session, add operation along 'DbComputer.dbnodes' will not pr

group_outputs []
group_outputs:  {}


{'execution_count': <Int: uuid: 14eeea98-1270-4e8d-9ff8-4afac350fa00 (pk: 8411) value: 1>}